### Import essential libraries

In [1]:
import more_utils
more_utils.set_logging_level("INFO")

from IPython.display import display
from more_utils.persistence import ModelarDB
from more_utils.service import TimeseriesService

### Create a connection to ModelarDB

Params:
1. hostname: ip adrress of the system where ModelarDB is running
2. interface: arrow (recommended), socket, http

In [2]:
conn_obj = ModelarDB.connect(hostname="localhost", interface="arrow")

### Create a timeseries service instance
Service instance can be used to access all the timeseries APIs.

In [3]:
ts_service = TimeseriesService(source_db_conn=conn_obj)

### API: get_time_series_data_from_ts_ids

#### 1. Fetch time-series data points for time series ids in `ts_ids`.

Params:
- ts_ids (List[int]): time series id(s).
- from_date (Union[str, None], optional): Start timestamp.
                                        Defaults to None.
- to_date (Union[str, None], optional): End timestamp.
                                        Defaults to None.
- merge_on (str, optional): common field to merge multiple
                            timeseries. Defaults to TIMESTAMP_LABEL.
- value_column_labels (Union[List[str], None], optional): List of
string to replace value column labels. Defaults to None.
- limit (Union[int, None], optional): No of data points to fetch.
                                    Defaults to None.

In [4]:
decompressed_ts = ts_service.get_time_series_data_from_ts_ids(ts_ids=[1], from_date='2019-01-01 00:00:02.0', to_date='2019-01-01 00:01:38', limit=5)

#### 2. Retrieve data once time series object is created.

There are two ways to retireve data,
- fetch_all: retrieve all data at once
- fetch_one: retrieve data a batch at a time

Time series data can be fetched in three dataframes,
- JSON (fetch_type="json")
- Pandas (fetch_type="pandas")
- PySpark (fetch_type="spark")


In [5]:
json_str = decompressed_ts.fetch_all(fetch_type="json")
print(json_str)

{
    "columns": [
        "TID",
        "TIMESTAMP",
        "VALUE_1"
    ],
    "data": [
        [
            1,
            "2019-01-01 00:00:02",
            404.89776611328125
        ],
        [
            1,
            "2019-01-01 00:00:04",
            406.8476867675781
        ],
        [
            1,
            "2019-01-01 00:00:06",
            426.9863586425781
        ],
        [
            1,
            "2019-01-01 00:00:08",
            440.9736633300781
        ],
        [
            1,
            "2019-01-01 00:00:10",
            472.5389099121094
        ]
    ]
}


In [6]:
df = decompressed_ts.fetch_all(fetch_type="pandas")
df.head(5)

,TID,TIMESTAMP,VALUE_1
0,1,2019-01-01 00:00:02,404.897766
1,1,2019-01-01 00:00:04,406.847687
2,1,2019-01-01 00:00:06,426.986359
3,1,2019-01-01 00:00:08,440.973663
4,1,2019-01-01 00:00:10,472.538910


In [7]:
df = decompressed_ts.fetch_all(fetch_type="spark")
df.show(5)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/24 10:20:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+-------------------+------------------+
|TID|          TIMESTAMP|           VALUE_1|
+---+-------------------+------------------+
|  1|2019-01-01 00:00:02|404.89776611328125|
|  1|2019-01-01 00:00:04| 406.8476867675781|
|  1|2019-01-01 00:00:06| 426.9863586425781|
|  1|2019-01-01 00:00:08| 440.9736633300781|
|  1|2019-01-01 00:00:10| 472.5389099121094|
+---+-------------------+------------------+



#### 3. If there are multiple time-series id, the final output is a merged(by timestamp column) time-series.

In [8]:
merged_time_series = ts_service.get_time_series_data_from_ts_ids(ts_ids=[1,2,3,4,5,6], value_column_labels=["active power","nacelle direction","pitch angle","rotor speed","wind direction","wind speed"], limit=10)

In [9]:
for ts_data in merged_time_series.fetch_next(batch_size=5):
    display(ts_data.head())

,TIMESTAMP,active power,nacelle direction,pitch angle,rotor speed,wind direction,wind speed
0,2019-01-01 00:00:02,404.897766,222.756058,0.0,11.350214,232.658554,5.069616
1,2019-01-01 00:00:04,406.847687,222.756058,0.0,11.380639,237.539307,4.918308
2,2019-01-01 00:00:06,426.986359,222.756058,0.0,11.411064,235.136139,5.506491
3,2019-01-01 00:00:08,440.973663,222.756058,0.0,11.441490,223.182190,5.672853
4,2019-01-01 00:00:10,472.538910,222.756058,0.0,11.471915,225.316437,5.942164


,TIMESTAMP,active power,nacelle direction,pitch angle,rotor speed,wind direction,wind speed
0,2019-01-01 00:00:12,491.533234,222.756058,0.0,11.502340,233.563400,5.740467
1,2019-01-01 00:00:14,499.575226,222.756058,0.0,11.532765,229.546890,5.827747
2,2019-01-01 00:00:16,518.149292,222.756058,0.0,11.563190,227.083176,5.956445
3,2019-01-01 00:00:18,530.716003,222.756058,0.0,11.593616,225.745743,5.796626
4,2019-01-01 00:00:20,545.869080,222.756058,0.0,11.624042,230.552887,5.788846


### API: get_time_series_data_models_from_ts_ids

#### 1. fetch data models generated from raw time series data

Params:
- ts_ids (List[int]): time series id(s).
- from_date (Union[str, None], optional): Start timestamp.
                                        Defaults to None.
- to_date (Union[str, None], optional): End timestamp.
                                        Defaults to None.
- limit (Union[int, None], optional): No of data points to fetch.
                                    Defaults to None.

In [10]:
ts_data_models = ts_service.get_time_series_data_models_from_ts_ids([1,2], limit=2)

In [11]:
df = ts_data_models.fetch_all(fetch_type="pandas")
df.head()

,TID,START_TIME,END_TIME,MTID,MODEL,OFFSETS
0,1,2019-01-01 00:00:02,2019-01-01 00:01:40,4,"b""C\xcar\xea\xde\x8cy\xafY\xfe\x12\xf6\x1f _R\...",b'\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00...
1,1,2019-01-01 00:01:42,2019-01-01 00:03:20,4,b'C\xd5\x1b\x89\xe0v\xfc;\x14\xcfy\x99x\x81\xa...,b'\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00...
2,2,2019-01-01 00:00:02,2019-01-01 11:45:48,2,b'C^\xc1\x8d',b'\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00...
3,2,2019-01-01 11:45:50,2019-01-01 11:47:14,3,b'?!\xda\xe9\x8b\x16\xc3\xd4\xc1\xa9\x1c{\x84\...,b'\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00...
